In [1]:
# Importing libraries
from netCDF4 import Dataset
from netCDF4 import MFDataset
import netCDF4 as nc
import os
import glob
import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cm
import matplotlib.dates as mdates
from matplotlib import rcParams
from matplotlib.colors import LinearSegmentedColormap, LogNorm
import re 
import pylag
from shapely.geometry import Point
import contextily as ctx
import datetime
import dask
import dask.dataframe as dd
import gc 
import concurrent.futures
import cftime  # Import cftime to handle cftime.DatetimeGregorian objects
from datetime import datetime

In [22]:
#define a path to the FVCOME data directory /home/samin/data/FVCOME_OUTPUT/Gldata/FVCOMEDATA
# go forlder 202301 and red nc file contain *2230101.nc
# define the path to the data directory
data_dir = '/home/abolmaal/data/FVCOME_OUTPUT/Simulations/Huron/output'
def sort_key(file):
    filename = os.path.basename(file)
    number = int(filename.split('__')[1].split('.')[0])
    return number
files = glob.glob(data_dir + "/Fvcome_huron_estuary_2023_Winter_*.nc")
files.sort(key=sort_key)

In [23]:
# Loop through each file, load it, create group_number as an integer, and save the updated NetCDF file
for file in files:
    # Read the NetCDF file
    ds = xr.open_dataset(file)

    # Convert the NetCDF 'group_id' variable to a DataFrame for processing
    netcdf_df = ds['group_id'].to_dataframe().reset_index()

    # Step 1: Count occurrences of each group_id in the NetCDF file
    netcdf_df['group_number'] = netcdf_df.groupby('group_id').cumcount()

    # Step 2: Combine 'group_id' and 'group_number' to create an integer (e.g., group_id=10 and count=1 would result in 101)
    netcdf_df['group_number'] = netcdf_df.apply(lambda row: int(f"{int(row['group_id'])}{row['group_number']}"), axis=1)

    # Step 3: Add the updated group_number column back into the NetCDF dataset
    ds['group_number'] = (('dim_0'), netcdf_df['group_number'].values)  # Replace 'dim_0' with the correct dimension

    # Step 4: Save the updated NetCDF file
    output_file_path = os.path.join(data_dir, f"updated_{os.path.basename(file)}")
    ds.to_netcdf(output_file_path)

    print(f"Updated NetCDF file saved: {output_file_path}")

Updated NetCDF file saved: /home/abolmaal/data/FVCOME_OUTPUT/Simulations/Huron/output/updated_Fvcome_huron_estuary_2023_Winter_Jan__1.nc
Updated NetCDF file saved: /home/abolmaal/data/FVCOME_OUTPUT/Simulations/Huron/output/updated_Fvcome_huron_estuary_2023_Winter_Feb__2.nc
Updated NetCDF file saved: /home/abolmaal/data/FVCOME_OUTPUT/Simulations/Huron/output/updated_Fvcome_huron_estuary_2023_Winter_Mar__3.nc
Updated NetCDF file saved: /home/abolmaal/data/FVCOME_OUTPUT/Simulations/Huron/output/updated_Fvcome_huron_estuary_2023_Winter_Apr__4.nc
Updated NetCDF file saved: /home/abolmaal/data/FVCOME_OUTPUT/Simulations/Huron/output/updated_Fvcome_huron_estuary_2023_Winter_May__5.nc
Updated NetCDF file saved: /home/abolmaal/data/FVCOME_OUTPUT/Simulations/Huron/output/updated_Fvcome_huron_estuary_2023_Winter_Jun__6.nc
Updated NetCDF file saved: /home/abolmaal/data/FVCOME_OUTPUT/Simulations/Huron/output/updated_Fvcome_huron_estuary_2023_Winter_July__7.nc
Updated NetCDF file saved: /home/abolmaa

In [27]:
# Get the output files and sort them using the sort_key function
files = glob.glob(os.path.join(data_dir, 'updated_Fvcome_huron_estuary_2023_Winter_*.nc'))
files.sort(key=sort_key)

# Open multiple NetCDF datasets with chunks using xarray
datasets = [xr.open_mfdataset(file) for file in files]

# (Optional) Print the sorted file names to verify the sorting
for file in files:
    print(file)

/home/abolmaal/data/FVCOME_OUTPUT/Simulations/Huron/output/updated_Fvcome_huron_estuary_2023_Winter_Jan__1.nc
/home/abolmaal/data/FVCOME_OUTPUT/Simulations/Huron/output/updated_Fvcome_huron_estuary_2023_Winter_Feb__2.nc
/home/abolmaal/data/FVCOME_OUTPUT/Simulations/Huron/output/updated_Fvcome_huron_estuary_2023_Winter_Mar__3.nc
/home/abolmaal/data/FVCOME_OUTPUT/Simulations/Huron/output/updated_Fvcome_huron_estuary_2023_Winter_Apr__4.nc
/home/abolmaal/data/FVCOME_OUTPUT/Simulations/Huron/output/updated_Fvcome_huron_estuary_2023_Winter_May__5.nc
/home/abolmaal/data/FVCOME_OUTPUT/Simulations/Huron/output/updated_Fvcome_huron_estuary_2023_Winter_Jun__6.nc
/home/abolmaal/data/FVCOME_OUTPUT/Simulations/Huron/output/updated_Fvcome_huron_estuary_2023_Winter_July__7.nc
/home/abolmaal/data/FVCOME_OUTPUT/Simulations/Huron/output/updated_Fvcome_huron_estuary_2023_Winter_Aug__8.nc
/home/abolmaal/data/FVCOME_OUTPUT/Simulations/Huron/output/updated_Fvcome_huron_estuary_2023_Winter_Sep__9.nc
/home/abo

In [28]:
#get the last output file in the dataset 
ds = datasets[-1]
#get the particle id index 
group_id = ds['group_id'].values
#get lat, lon and time for the last time step 
lat = ds['latitude'].values
lon = ds['longitude'].values
lon = np.where(lon > 180, lon - 360, lon)  # Convert longitude values to -180 to 180 range
time = ds['time'].values
group_number = ds['group_number'].values

In [29]:
ds

<xarray.Dataset>
Dimensions:                   (time: 121, particles: 4950, dim_0: 4950)
Coordinates:
  * time                      (time) datetime64[ns] 2023-12-01 ... 2023-12-31
Dimensions without coordinates: particles, dim_0
Data variables:
    group_id                  (particles) int32 dask.array<chunksize=(4950,), meta=np.ndarray>
    longitude                 (time, particles) float32 dask.array<chunksize=(121, 4950), meta=np.ndarray>
    latitude                  (time, particles) float32 dask.array<chunksize=(121, 4950), meta=np.ndarray>
    depth                     (time, particles) float32 dask.array<chunksize=(121, 4950), meta=np.ndarray>
    host_fvcom                (time, particles) int32 dask.array<chunksize=(121, 4950), meta=np.ndarray>
    error_status              (time, particles) int32 dask.array<chunksize=(121, 4950), meta=np.ndarray>
    in_domain                 (time, particles) int32 dask.array<chunksize=(121, 4950), meta=np.ndarray>
    is_beached                (time, particles) int32 dask.array<chunksize=(121, 4950), meta=np.ndarray>
    land_boundary_encounters  (time, particles) int32 dask.array<chunksize=(121, 4950), meta=np.ndarray>
    thetao                    (time, particles) float32 dask.array<chunksize=(121, 4950), meta=np.ndarray>
    so                        (time, particles) float32 dask.array<chunksize=(121, 4950), meta=np.ndarray>
    group_number              (dim_0) int64 dask.array<chunksize=(4950,), meta=np.ndarray>
Attributes:
    title:    PyLag -- Plymouth Marine Laboratory

In [30]:

# Assuming lon, lat, and group_id are numpy arrays or pandas Series
# Check the lengths of the arrays
print(f"Length of lon: {len(lon)}")
print(f"Length of lat: {len(lat)}")
print(f"Length of group_id: {len(group_id)}")

# Ensure the arrays have the same length
if len(lon) == len(lat) == len(group_id):
    # Plot the particles where group_id = 1
    plt.figure(figsize=(10, 6))
    plt.scatter(lon[group_id == 1], lat[group_id == 1], c='r', s=1)
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.title('Particles with group_id = 1')
    plt.show()
else:
    print("The lengths of lon, lat, and group_id arrays do not match.")

Length of lon: 121
Length of lat: 121
Length of group_id: 4950
The lengths of lon, lat, and group_id arrays do not match.


In [20]:
print (group_number)

['000' '001' '002' ... '14745' '14746' '14747']


In [31]:
# dss

array(['2023-12-01T00:00:00.000000000', '2023-12-01T06:00:00.000000000',
       '2023-12-01T12:00:00.000000000', '2023-12-01T18:00:00.000000000',
       '2023-12-02T00:00:00.000000000', '2023-12-02T06:00:00.000000000',
       '2023-12-02T12:00:00.000000000', '2023-12-02T18:00:00.000000000',
       '2023-12-03T00:00:00.000000000', '2023-12-03T06:00:00.000000000',
       '2023-12-03T12:00:00.000000000', '2023-12-03T18:00:00.000000000',
       '2023-12-04T00:00:00.000000000', '2023-12-04T06:00:00.000000000',
       '2023-12-04T12:00:00.000000000', '2023-12-04T18:00:00.000000000',
       '2023-12-05T00:00:00.000000000', '2023-12-05T06:00:00.000000000',
       '2023-12-05T12:00:00.000000000', '2023-12-05T18:00:00.000000000',
       '2023-12-06T00:00:00.000000000', '2023-12-06T06:00:00.000000000',
       '2023-12-06T12:00:00.000000000', '2023-12-06T18:00:00.000000000',
       '2023-12-07T00:00:00.000000000', '2023-12-07T06:00:00.000000000',
       '2023-12-07T12:00:00.000000000', '2023-12-07